<a href="https://colab.research.google.com/github/bahag-hammers/GoogleLab/blob/main/%5BDatathon%5D_Lab_1_Data_Hackathon_Setup_%26_Preparation_%5BMAKE_A_COPY%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1:** Datathon Preparation
This notebooks sets up the enrvironment and prepares the datasets from the original events data

**Please copy this CoLab**

### **Step 1:** Parameters and Authentication
Authenticate yourself against Google Cloud Platform. In Colab there is a seperate python lib for that

In [ ]:
#@title Parameters

project_id = "" #@param {type:"string"}
team_name  = "" #@param {type:"string"}

location   = "us" #This is necessary as the original dataset is located there.
dataset_name = "datathon_ds_{}".format(team_name)


In [ ]:
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()

client = bigquery.Client(project=project_id)

### **Step 2:** Create the Team Dataset
Create a dataset for your team. We do using the command line bq command that is installed in a CoLab.

In [ ]:
!bq mk --location=$location --dataset $project_id:$dataset_name

Let's check if the dataset is existing. Just list all the datasets in the project. We can do this also via the BQ python library:

In [ ]:
for dataset in client.list_datasets():
  print(dataset.dataset_id)

### **Step 3:** Test Access to Raw Dataset
Check if the access to the dataset is working properly. This time we use BigQuery magic (pre-installed in CoLab and Vertex Workbenchs) to execute a BigQuery query directly from here:

In [ ]:
%%bigquery --project $project_id

SELECT *
FROM `firebase-public-project.analytics_153293282.events_*`
LIMIT 3

### **Step 4:** Data Preparation
Now we prepare some Views on top of the dataset. These views already execute some data preparations and feature engineering tasks. We will come back to the details of this later.

**View 1** analyzes the user and its first and latest appearance.

In [ ]:
#@title Create view user_returninginfo

view_id  = "{}.{}.user_returninginfo".format(project_id, dataset_name)
view     = bigquery.Table(view_id)

view.view_query = f'''
WITH firstlasttouch AS (
    SELECT
      user_pseudo_id,
      MIN(event_timestamp) AS user_first_engagement,
      MAX(event_timestamp) AS user_last_engagement
    FROM
      `firebase-public-project.analytics_153293282.events_*`
    WHERE event_name="user_engagement"
    GROUP BY
      user_pseudo_id

  )
  SELECT
    user_pseudo_id,
    user_first_engagement,
    user_last_engagement,
    EXTRACT(MONTH from TIMESTAMP_MICROS(user_first_engagement)) as month,
    EXTRACT(DAYOFYEAR from TIMESTAMP_MICROS(user_first_engagement)) as julianday,
    EXTRACT(DAYOFWEEK from TIMESTAMP_MICROS(user_first_engagement)) as dayofweek,

    #add 24 hr to user's first touch
    (user_first_engagement + 86400000000) AS ts_24hr_after_first_engagement,

#churned = 1 if last_touch within 24 hr of app installation, else 0
IF (user_last_engagement < (user_first_engagement + 86400000000),
    1,
    0 ) AS churned,

#bounced = 1 if last_touch within 10 min, else 0
IF (user_last_engagement <= (user_first_engagement + 600000000),
    1,
    0 ) AS bounced,
  FROM
    firstlasttouch
  GROUP BY
    1,2,3

'''

# Create the view
view = client.create_table(view, exists_ok=True)
print(f"Created {view.table_type}: {str(view.reference)}")


**View 2** aggregates the demographics of the users.

In [ ]:
#@title Create view user_demographics

view_id  = "{}.{}.user_demographics".format(project_id, dataset_name)
view     = bigquery.Table(view_id)

view.view_query = f'''
  WITH first_values AS (
      SELECT
          user_pseudo_id,
          geo.country as country,
          device.operating_system as operating_system,
          device.language as language,
          ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp DESC) AS row_num
      FROM `firebase-public-project.analytics_153293282.events_*`
      WHERE event_name="user_engagement"
      )
  SELECT * EXCEPT (row_num)
  FROM first_values
  WHERE row_num = 1
'''

# Create the view
view = client.create_table(view, exists_ok=True)
print(f"Created {view.table_type}: {str(view.reference)}")


**View 3** creates a aggregated user behaviour view

In [ ]:
#@title Create view user_aggregate_behavior

view_id  = "{}.{}.user_aggregate_behavior".format(project_id, dataset_name)
view     = bigquery.Table(view_id)

view.view_query = f'''
WITH events_first24hr AS (
    #select user data only from first 24 hr of using the app
    SELECT
      e.*
    FROM
      `firebase-public-project.analytics_153293282.events_*` e
    JOIN
      {dataset_name}.user_returninginfo r
    ON
      e.user_pseudo_id = r.user_pseudo_id
    WHERE
      e.event_timestamp <= r.ts_24hr_after_first_engagement
    )
SELECT
  user_pseudo_id,
  SUM(IF(event_name = 'user_engagement', 1, 0)) AS cnt_user_engagement,
  SUM(IF(event_name = 'level_start_quickplay', 1, 0)) AS cnt_level_start_quickplay,
  SUM(IF(event_name = 'level_end_quickplay', 1, 0)) AS cnt_level_end_quickplay,
  SUM(IF(event_name = 'level_complete_quickplay', 1, 0)) AS cnt_level_complete_quickplay,
  SUM(IF(event_name = 'level_reset_quickplay', 1, 0)) AS cnt_level_reset_quickplay,
  SUM(IF(event_name = 'post_score', 1, 0)) AS cnt_post_score,
  SUM(IF(event_name = 'spend_virtual_currency', 1, 0)) AS cnt_spend_virtual_currency,
  SUM(IF(event_name = 'ad_reward', 1, 0)) AS cnt_ad_reward,
  SUM(IF(event_name = 'challenge_a_friend', 1, 0)) AS cnt_challenge_a_friend,
  SUM(IF(event_name = 'completed_5_levels', 1, 0)) AS cnt_completed_5_levels,
  SUM(IF(event_name = 'use_extra_steps', 1, 0)) AS cnt_use_extra_steps,
FROM
  events_first24hr
GROUP BY
  1
'''

# Create the view
view = client.create_table(view, exists_ok=True)
print(f"Created {view.table_type}: {str(view.reference)}")

**View 4** combines all the other views into a single flat view for training purposes.

In [ ]:
#@title Create view for the full dataset
view_id  = "{}.{}.cc_full_dataset".format(project_id, dataset_name)
view     = bigquery.Table(view_id)

view.view_query = f'''

  SELECT
    dem.*,
    IFNULL(beh.cnt_user_engagement, 0) AS cnt_user_engagement,
    IFNULL(beh.cnt_level_start_quickplay, 0) AS cnt_level_start_quickplay,
    IFNULL(beh.cnt_level_end_quickplay, 0) AS cnt_level_end_quickplay,
    IFNULL(beh.cnt_level_complete_quickplay, 0) AS cnt_level_complete_quickplay,
    IFNULL(beh.cnt_level_reset_quickplay, 0) AS cnt_level_reset_quickplay,
    IFNULL(beh.cnt_post_score, 0) AS cnt_post_score,
    IFNULL(beh.cnt_spend_virtual_currency, 0) AS cnt_spend_virtual_currency,
    IFNULL(beh.cnt_ad_reward, 0) AS cnt_ad_reward,
    IFNULL(beh.cnt_challenge_a_friend, 0) AS cnt_challenge_a_friend,
    IFNULL(beh.cnt_completed_5_levels, 0) AS cnt_completed_5_levels,
    IFNULL(beh.cnt_use_extra_steps, 0) AS cnt_use_extra_steps,
    ret.user_first_engagement,
    ret.churned
  FROM
    {dataset_name}.user_returninginfo ret
  LEFT OUTER JOIN
    {dataset_name}.user_demographics dem
  ON
    ret.user_pseudo_id = dem.user_pseudo_id
  LEFT OUTER JOIN
    {dataset_name}.user_aggregate_behavior beh
  ON
    ret.user_pseudo_id = beh.user_pseudo_id
  WHERE ret.bounced = 0
'''

# Create the view
view = client.create_table(view, exists_ok=True)
print(f"Created {view.table_type}: {str(view.reference)}")

## Step 5: Train & Test Split
Split the dataset into a train & test dataset using views and a stable algorithm

In [ ]:
#@title Create view train_dataset
view_id  = "{}.{}.cc_train_dataset".format(project_id, dataset_name)
view     = bigquery.Table(view_id)

view.view_query = f'''

  SELECT * FROM {dataset_name}.cc_full_dataset
  WHERE ABS(MOD(FARM_FINGERPRINT(user_pseudo_id), 10)) < 8

'''

# Create the view
view = client.create_table(view, exists_ok=True)
print(f"Created {view.table_type}: {str(view.reference)}")

In [ ]:
#@title Create view test dataset
view_id  = "{}.{}.cc_eval_dataset".format(project_id, dataset_name)
view     = bigquery.Table(view_id)

view.view_query = f'''

  SELECT * FROM {dataset_name}.cc_full_dataset
  WHERE ABS(MOD(FARM_FINGERPRINT(user_pseudo_id), 10)) = 8

'''

# Create the view
view = client.create_table(view, exists_ok=True)
print(f"Created {view.table_type}: {str(view.reference)}")

## **Finally**
This is the dataset we have prepared now:
To remeber: This is how the final training dataset looks like

| User Demographic Data | User Behavioral Data | Label
| --- | --- | --- |
| country | cnt_user_engagement | churned
| operating_system | cnt_level_start_quickplay |  |
| language | cnt_level_end_quickplay |  |
|  |  cnt_level_complete_quickplay |  |
|  |  cnt_level_reset_quickplay |  |
|  |  cnt_post_score |  |
|  |  cnt_spend_virtual_currency |  |
|  |  cnt_ad_reward |  |
|  |  cnt_challenge_a_friend |  |
|  |  cnt_completed_5_levels |  |
|  |  cnt_use_extra_steps |  |
|  |  user_first_engagement |  |

### Cleanup
Call this an the entire dataset will be cleaned up

In [ ]:
# Uncomment this to delete the dataset
#!bq rm -f --location=$location --dataset $project_id:$dataset_name